# Lab 4: Deploy Multi-Agent System to Production with AgentCore Runtime

## Overview

In this lab, you will deploy your Multi-Agent Customer Support System to production using **Amazon Bedrock AgentCore Runtime** with comprehensive observability and monitoring.

[Amazon Bedrock AgentCore Runtime](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/agents-tools-runtime.html) is a secure, fully managed runtime that empowers organizations to deploy and scale AI agents in production, regardless of framework, protocol, or model choice.

**Workshop Journey:**
- **Lab 1 (Done)**: Multi-Agent Foundation - Built orchestrator with specialized agents
- **Lab 2 (Done)**: Multi-Agent Memory - Added persistent memory across agents
- **Lab 3 (Done)**: Multi-Agent Gateway - Secure tool sharing via AgentCore Gateway
- **Lab 4 (Current)**: Multi-Agent Runtime - Deploy with observability and monitoring
- **Lab 5**: Multi-Agent Testing - Post your queries to test multi agent routing

### Why AgentCore Runtime & Production Deployment Matter

**Current State (Labs 1-3):** Multi-agent system runs locally with centralized tools but faces production challenges:
- Agents run locally in single sessions
- No comprehensive monitoring or debugging capabilities
- Cannot handle multiple concurrent users reliably
- No automatic scaling or error recovery

**After this lab:** Production-ready multi-agent infrastructure with:
- Serverless auto-scaling for each agent type
- Comprehensive observability with traces, metrics, and logging
- Enterprise reliability with automatic error recovery
- Secure deployment with proper access controls
- Independent scaling of orchestrator and specialized agents

### Adding Comprehensive Observability with AgentCore Observability

[AgentCore Observability](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/observability.html) provides full visibility into your multi-agent system's behavior in production, including:
- Request tracing across agent interactions
- Performance monitoring for each agent type
- Tool usage patterns and memory access analytics
- Cross-agent conversation flow analysis

## Multi-Agent Runtime Architecture for Lab 4

All agents now run in AgentCore Runtime with full observability through CloudWatch. The orchestrator coordinates requests across specialized agents, each with independent scaling and monitoring. Memory and Gateway integrations from previous labs remain fully functional.

### Key Features
- **Multi-Agent Deployment:** Deploy orchestrator and specialized agents independently
- **Comprehensive Observability:** Full request tracing across agent interactions
- **Independent Scaling:** Each agent type scales based on its specific demand
- **Production Reliability:** Enterprise-grade error handling and recovery

## Prerequisites

* Python 3.12+
* AWS account with appropriate permissions
* Docker, Finch or Podman installed and running
* Amazon Bedrock AgentCore SDK
* Strands Agents framework
* Complete Labs 1-3 (Multi-Agent System, Memory, Gateway)

**Note**: You MUST enable [CloudWatch Transaction Search](https://docs.aws.amazon.com/AmazonCloudWatch/latest/monitoring/Enable-TransactionSearch.html) to see AgentCore Observability traces.

## Step 1: Import Required Libraries and Verify Prerequisites

Ensure you have enabled [CloudWatch Transaction Search](https://docs.aws.amazon.com/AmazonCloudWatch/latest/monitoring/Enable-TransactionSearch.html) to see AgentCore Observability traces. You can follow the console steps.

In [ ]:
# Import required libraries
import os
import json
import boto3
import time
from bedrock_agentcore_starter_toolkit import Runtime
from lab_helpers.utils import (
    get_ssm_parameter,
    put_ssm_parameter,
    create_agentcore_runtime_execution_role,
    setup_or_reuse_cognito_user_pool,
    reauthenticate_user
)
from lab_helpers.lab2_multi_agent_memory import create_or_get_multi_agent_memory

print("✅ Libraries imported successfully")

# Verify memory resource exists from Lab 2
try:
    memory_id = create_or_get_multi_agent_memory()
    print(f"✅ Multi-agent memory verified: {memory_id}")
except Exception as e:
    print(f"❌ Memory verification failed: {e}")
    print("Please ensure Lab 2 (Multi-Agent Memory) has been completed")

## Step 2: Configure Authentication for Multi-Agent Runtime

Set up Amazon Cognito authentication that will be shared across all agents in our multi-agent system.

In [ ]:
existing_pool_id = get_ssm_parameter("/app/reinvent/agentcore/userpool_id")
print(existing_pool_id)
existing_client_id = get_ssm_parameter("/app/reinvent/agentcore/machine_client_id")
print(existing_client_id)
existing_discovery_url = get_ssm_parameter("/app/reinvent/agentcore/cognito_discovery_url")
print(existing_discovery_url)

In [ ]:
print("Setting up Amazon Cognito user pool for multi-agent system...")
cognito_config = setup_or_reuse_cognito_user_pool()
print("✅ Cognito setup completed")
print(f"Client ID: {cognito_config.get('ClientId')}")  # Changed: 'client_id' → 'ClientId'
print(f"Discovery URL: {cognito_config.get('discovery_url')}")  # This one is correct


In [ ]:
# Initialize region and memory resource
import boto3
from bedrock_agentcore.memory import MemoryClient
from lab_helpers.utils import get_ssm_parameter, put_ssm_parameter

REGION = boto3.session.Session().region_name
memory_client = MemoryClient(region_name=REGION)

try:
    # Try to get existing memory ID
    memory_id = get_ssm_parameter("/app/reinvent/agentcore/memory_id")
    print(f"✅ Found existing memory ID: {memory_id}")
except:
    # Create new memory resource (name must be alphanumeric + underscores only)
    print("Creating new memory resource...")
    response = memory_client.create_memory(
        name="reinvent_customer_support_memory",
        description="Memory for Reinvent Customer Support Multi-Agent System"
    )
    memory_id = response["id"]
    put_ssm_parameter("/app/reinvent/agentcore/memory_id", memory_id)
    print(f"✅ Created memory resource: {memory_id}")

# Verify memory exists
try:
    memory_client.get_memory(memoryId=memory_id)
    print(f"✅ Memory resource verified: {memory_id}")
except Exception as e:
    print(f"❌ Memory verification failed: {e}")


# CUSTOMER AGENT DEPLOYMENT

## Step 3: Deploy Customer Support Agent to AgentCore Runtime

The customer support agent handles specific customer service queries with access to gateway tools from Lab 3.

**A2A Protocol:** This agent is deployed with Agent-to-Agent (A2A) protocol support, allowing it to communicate with other agents using standardized agent cards and message passing.

**Note** Since this is not a production environment, please ignore Platform mismatch warnings throughout this lab.

In [ ]:
# Initialize runtime toolkit
boto_session = boto3.session.Session()
region = boto_session.region_name

# Create execution role for customer support agent
customer_support_execution_role = create_agentcore_runtime_execution_role(agent_name="customer_support")
print(f"✅ Customer support execution role: {customer_support_execution_role}")

# Configure customer support runtime with A2A protocol
customer_support_runtime = Runtime()
customer_support_config = customer_support_runtime.configure(
    entrypoint="lab_helpers/runtime/customer_support_runtime.py",
    execution_role=customer_support_execution_role,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name="customer_support_agent",
    protocol="A2A",  # Enable Agent-to-Agent protocol
    authorizer_configuration={
        "customJWTAuthorizer": {
            "allowedClients": [cognito_config.get('ClientId')],  # Use current Cognito
            "discoveryUrl": cognito_config.get('discovery_url'),  # Use current Cognito
        }
    },
)
print("✅ Customer support agent configured for deployment with A2A protocol")

print("🚀 Launching customer support agent to AgentCore Runtime...")
customer_support_launch = customer_support_runtime.launch()
customer_support_arn = customer_support_launch.agent_arn

# Store customer support ARN
put_ssm_parameter("/app/reinvent/agentcore/customer_support_arn", customer_support_arn)
print(f"✅ Customer support agent deployed: {customer_support_arn}")


# KB AGENT DEPLOYMENT

## Step 4: Deploy Knowledge Base Agent to AgentCore Runtime

The knowledge base agent handles complex queries requiring access to documentation and knowledge repositories.

**A2A Protocol:** This agent is deployed with Agent-to-Agent (A2A) protocol support, allowing it to be invoked by the orchestrator agent.

In [ ]:
# Create execution role for knowledge base agent
knowledge_base_execution_role = create_agentcore_runtime_execution_role(agent_name="knowledge_base")
print(f"✅ Knowledge base execution role: {knowledge_base_execution_role}")

# Configure knowledge base runtime with A2A protocol
knowledge_base_runtime = Runtime()
knowledge_base_config = knowledge_base_runtime.configure(
    entrypoint="lab_helpers/runtime/knowledge_base_runtime.py",
    execution_role=knowledge_base_execution_role,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name="knowledge_base_agent",
    protocol="A2A",  # Enable Agent-to-Agent protocol
    authorizer_configuration={
        "customJWTAuthorizer": {
            "allowedClients": [cognito_config.get('ClientId')],  # Use current Cognito
            "discoveryUrl": cognito_config.get('discovery_url'),  # Use current Cognito
        }
    },
)

print("✅ Knowledge base agent configured for deployment with A2A protocol")

print("🚀 Launching knowledge base agent to AgentCore Runtime...")
knowledge_base_launch = knowledge_base_runtime.launch()
knowledge_base_arn = knowledge_base_launch.agent_arn

# Store knowledge base ARN
put_ssm_parameter("/app/reinvent/agentcore/knowledge_base_arn", knowledge_base_arn)
print(f"✅ Knowledge base agent deployed: {knowledge_base_arn}")


# ORCHESTRATOR AGENT DEPLOYMENT

## Step 5: Deploy Orchestrator Agent to AgentCore Runtime

The orchestrator agent coordinates requests across specialized agents and handles the main customer interaction flow.

**Architecture:** The orchestrator is a regular BedrockAgentCoreApp (NOT an A2A server). It acts as an A2A client, making direct HTTP calls with JSON-RPC 2.0 to invoke the customer_support and knowledge_base A2A servers. Users interact with the orchestrator using simple JSON payloads like `{"prompt": "your question"}`.

**Important:** The orchestrator is deployed LAST because it needs to route requests to the customer_support and knowledge_base agents, which must be deployed first so their ARNs are available in SSM Parameter Store.

In [ ]:
# Initialize runtime toolkit
boto_session = boto3.session.Session()
region = boto_session.region_name

# Create execution role for orchestrator
orchestrator_execution_role = create_agentcore_runtime_execution_role(agent_name="orchestrator")
print(f"✅ Orchestrator execution role: {orchestrator_execution_role}")

# Configure orchestrator runtime with A2A protocol
orchestrator_runtime = Runtime()
orchestrator_config = orchestrator_runtime.configure(
    entrypoint="lab_helpers/runtime/orchestrator_runtime.py",
    deployment_type="direct_code_deploy",
    runtime_type="PYTHON_3_13",
    execution_role=orchestrator_execution_role,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name="orchestrator_agent",
    # Orchestrator is a regular agent that connects to A2A servers
    authorizer_configuration={
        "customJWTAuthorizer": {
            "allowedClients": [cognito_config.get('ClientId')],  # Use current Cognito
            "discoveryUrl": cognito_config.get('discovery_url'),  # Use current Cognito
        }
    },
    request_header_configuration={
        "requestHeaderAllowlist": ["Authorization"]
    }
)
print("✅ Orchestrator agent configured for deployment (connects to A2A servers)")

print("🚀 Launching orchestrator agent to AgentCore Runtime...")
orchestrator_launch = orchestrator_runtime.launch(
    env_vars={
        "AGENTCORE_KNOWLEDGE_BASE_ARN": get_ssm_parameter("/app/reinvent/agentcore/knowledge_base_arn"),
        "AGENTCORE_CUSTOMER_SUPPORT_ARN": get_ssm_parameter("/app/reinvent/agentcore/customer_support_arn")
    }
)
orchestrator_arn = orchestrator_launch.agent_arn

# Store orchestrator ARN
put_ssm_parameter("/app/reinvent/agentcore/orchestrator_arn", orchestrator_arn)
print(f"✅ Orchestrator agent deployed: {orchestrator_arn}")

## Step 6: Test All Deployed Agents

Now that all agents are deployed, let's verify they're working correctly and can communicate with each other.

In [ ]:
# Test all deployed agents
print("\n" + "="*60)
print("🧪 TESTING ALL DEPLOYED AGENTS")
print("="*60)

import urllib.parse
import requests
from uuid import uuid4

# Update SSM parameters with current Cognito config
current_client_id = cognito_config.get('ClientId')
current_discovery_url = cognito_config.get('discovery_url')
current_bearer_token = cognito_config.get('Bearer Token')

put_ssm_parameter("/app/reinvent/agentcore/machine_client_id", current_client_id)
put_ssm_parameter("/app/reinvent/agentcore/cognito_discovery_url", current_discovery_url)

print(f"✅ Updated runtime config with Client ID: {current_client_id}")

# Test queries for each agent
agents = {
    "orchestrator": "My iPhone is not connecting with Bluetooth. What should I do?",
    "customer_support": "I need help with my warranty claim for a defective product.",
    "knowledge_base": "How do I address overheating?"
}

# A2A agents (customer_support and knowledge_base) require JSON-RPC 2.0 format
a2a_agents = ["customer_support", "knowledge_base"]

# Test each agent
for agent_type, query in agents.items():
    print(f"\n{'='*60}")
    print(f"🎯 TESTING {agent_type.upper()} AGENT")
    print("="*60)
    
    try:
        agent_arn = get_ssm_parameter(f"/app/reinvent/agentcore/{agent_type}_arn")
        
        if agent_arn:
            print(f"🤖 {agent_type.title()} ARN: {agent_arn}")
            print(f"📝 Query: {query}")
            
            encoded_arn = urllib.parse.quote(agent_arn, safe='')
            url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{encoded_arn}/invocations/"
            
            # Use A2A protocol (JSON-RPC 2.0) for A2A servers
            if agent_type in a2a_agents:
                message_id = str(uuid4())
                session_id = str(uuid4())
                
                payload = {
                    "jsonrpc": "2.0",
                    "id": message_id,
                    "method": "message/send",
                    "params": {
                        "message": {
                            "role": "user",
                            "parts": [
                                {
                                    "kind": "text",
                                    "text": query
                                }
                            ],
                            "messageId": message_id
                        }
                    }
                }
                
                headers = {
                    "Authorization": f"Bearer {current_bearer_token}",
                    "X-Amzn-Bedrock-AgentCore-Runtime-Session-Id": session_id,
                    "Content-Type": "application/json"
                }
            else:
                # Orchestrator uses simple prompt format
                payload = {"prompt": query}
                headers = {
                    "Authorization": f"Bearer {current_bearer_token}",
                    "Content-Type": "application/json"
                }
            
            response = requests.post(url, json=payload, headers=headers)
            
            if response.status_code == 200:
                print(f"✅ {agent_type.title()} Response:")
                result = response.json()
                
                # Extract text from A2A response format
                if agent_type in a2a_agents and "result" in result:
                    if "artifacts" in result["result"]:
                        for artifact in result["result"]["artifacts"]:
                            if "parts" in artifact:
                                for part in artifact["parts"]:
                                    if part.get("kind") == "text":
                                        print(part.get("text", ""))
                else:
                    print(result)
            else:
                print(f"❌ Request failed: {response.status_code} - {response.text}")
                
        else:
            print(f"❌ {agent_type.title()} agent not found")
            
    except Exception as e:
        print(f"❌ Error testing {agent_type}: {e}")

print(f"\n{'='*60}")
print("🎉 MULTI-AGENT TESTING COMPLETE")
print("="*60)


In [ ]:
# Multi-Agent Continuity Check with Responses
print("\n" + "="*60)
print("🔄 MULTI-AGENT CONTINUITY CHECK WITH RESPONSES")
print("="*60)

import urllib.parse
import requests
import time

# Get current authentication
current_bearer_token = cognito_config.get('Bearer Token')

# Continuity test scenarios
continuity_tests = {
    "orchestrator": [
        "Hello, I need help with my device",
        "My laptop won't turn on"
    ],
    "customer_support": [
        "What's your return policy?",
        "What are smartphones specifications?"
    ],
    "knowledge_base": [
        "How do I troubleshoot connectivity issues?"
    ]
}

# A2A agents require JSON-RPC 2.0 format
a2a_agents = ["customer_support", "knowledge_base"]

# Track results
results = {}

for agent_type, queries in continuity_tests.items():
    print(f"\n{'='*50}")
    print(f"🧪 TESTING {agent_type.upper()} CONTINUITY")
    print("="*50)
    
    agent_results = []
    
    try:
        agent_arn = get_ssm_parameter(f"/app/reinvent/agentcore/{agent_type}_arn")
        encoded_arn = urllib.parse.quote(agent_arn, safe='')
        url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{encoded_arn}/invocations/"
        
        # Generate session ID for continuity testing
        session_id = str(uuid4())
        
        for i, query in enumerate(queries, 1):
            print(f"\n🔍 Test {i}: {query}")
            print("-" * 40)
            
            try:
                # Use A2A protocol for A2A servers
                if agent_type in a2a_agents:
                    message_id = str(uuid4())
                    
                    payload = {
                        "jsonrpc": "2.0",
                        "id": message_id,
                        "method": "message/send",
                        "params": {
                            "message": {
                                "role": "user",
                                "parts": [
                                    {
                                        "kind": "text",
                                        "text": query
                                    }
                                ],
                                "messageId": message_id
                            }
                        }
                    }
                    
                    headers = {
                        "Authorization": f"Bearer {current_bearer_token}",
                        "X-Amzn-Bedrock-AgentCore-Runtime-Session-Id": session_id,
                        "Content-Type": "application/json"
                    }
                else:
                    # Orchestrator uses simple prompt format
                    payload = {"prompt": query}
                    headers = {
                        "Authorization": f"Bearer {current_bearer_token}",
                        "Content-Type": "application/json"
                    }
                
                response = requests.post(
                    url,
                    json=payload,
                    headers=headers,
                    timeout=30
                )
                
                if response.status_code == 200:
                    response_data = response.json()
                    print(f"✅ SUCCESS")
                    
                    # Extract text from A2A response format
                    if agent_type in a2a_agents and "result" in response_data:
                        if "artifacts" in response_data["result"]:
                            for artifact in response_data["result"]["artifacts"]:
                                if "parts" in artifact:
                                    for part in artifact["parts"]:
                                        if part.get("kind") == "text":
                                            print(f"📝 Response: {part.get('text', '')}")
                    else:
                        print(f"📝 Response: {response_data}")
                    
                    agent_results.append("PASS")
                else:
                    print(f"❌ FAILED - Status: {response.status_code}")
                    print(f"📝 Error: {response.text}")
                    agent_results.append("FAIL")
                    
            except Exception as e:
                print(f"❌ ERROR - {str(e)}")
                agent_results.append("ERROR")
            
            time.sleep(2)  # Pause between requests
            
        results[agent_type] = agent_results
        
    except Exception as e:
        print(f"❌ Agent {agent_type} not available: {e}")
        results[agent_type] = ["NOT_DEPLOYED"]

# Summary Report
print(f"\n{'='*60}")
print("📊 CONTINUITY CHECK SUMMARY")
print("="*60)

for agent_type, test_results in results.items():
    passed = test_results.count("PASS")
    total = len(test_results)
    status = "✅ HEALTHY" if passed == total else "⚠️ ISSUES" if passed > 0 else "❌ FAILED"
    
    print(f"{agent_type.upper()}: {status} ({passed}/{total} tests passed)")

# Overall health
all_results = [result for results_list in results.values() for result in results_list]
total_passed = all_results.count("PASS")
total_tests = len(all_results)
overall_health = (total_passed / total_tests * 100) if total_tests > 0 else 0

print(f"\n🎯 OVERALL SYSTEM HEALTH: {overall_health:.1f}% ({total_passed}/{total_tests})")

if overall_health >= 90:
    print("🟢 System is fully operational")
elif overall_health >= 70:
    print("🟡 System has minor issues")
else:
    print("🔴 System needs attention")


## 🎉 Lab 4 Complete!

Congratulations! You've successfully deployed your multi-agent system to production using AgentCore Runtime. Your system now has enterprise-grade observability, monitoring, and scalability.

### ✅ What You Accomplished

- Deployed agents to AgentCore Runtime for production scaling
- Integrated CloudWatch observability and tracing
- Configured secure authentication with Cognito
- Tested multi-agent coordination in production environment
- Validated system health and continuity

### 📖 **Continue Your Journey**

**[→ Proceed to Lab 5: Multi-Agent Frontend](lab-05-multi-agent-frontend.ipynb)**